# Višeklasna klasifikacija korišćenjem neuronskih mreža

U ovoj svesci, naš zadatak će biti da napravimo klasfikator koji slici cifre pridružuje cifru koja se na njoj nalazi. Koristićemo poznati MNIST skup slika cifara i pristup koji je baziran na neuronskim mrežama. Koristićemo neuronsku mrežu sa propagacijom unapred, a već na sledećem času, u istom zadatku klasifikacije slika, uvešćemo konvolutivne neuronske mreže koje predstavlju prirodniji izbor. 

In [3]:
import numpy as np
from matplotlib import pyplot as plt

In [25]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import regularizers

In [5]:
np.random.seed(10)

## Korak 1: Učitavanje skupa podataka

MNIST je skup podataka koji sadrži slike rukom pisanih cifara. Slika ima ukupno 70.000 od čega se obično 60.000 slika koristi za treniranje, a preostalih 10.000 za testiranje. Sve slike u skupu podataka su dimenzije 28px x 28px i u crno-belom su modu. Više o MNIST skupu podataka, kao i rezultatima dobijenim prvim algoritmima mašinskog učenja, možete pročitati [ovde](http://yann.lecun.com/exdb/mnist/).

Da bismo učitali MNIST skup podataka iskoristićemo podršku Keras biblioteke preko `mnist.load_data()` funkcije. 

## Korak 2: Priprema podataka

Da bismo slike prosledili kao ulaze mreže, transformisaćemo ih u vektore dužine 28x28=784.

Vrednosti ciljne promenljive ćemo pripremiti tako što ćemo ih transformisati u vektore dužine 10 sa jedinicom na poziciji koja odgovara vrednosti cifre (takozvano *one-hot* kodiranje). Na primer, broj 3 će biti transformisan u vektor \[0, 0, 0, 1, 0, 0, 0, 0, 0, 0\] sa jedinicom na poziciji tri. 

Za ovu transformaciju iskoristićemo funkciju `to_categorical` paketa `keras.utils`.

## Korak 3: Pravljenje modela

Na slici je prikazana arhitekture mreže koju ćemo kreirati. 

<img src="assets/mlp_mnist.png" />

#### Ulazni sloj

Ulaz u mrežu je "ispravljena" slika dimenzije 784 piksela.

<img src="assets/mnist-input.png" />


#### Skriveni slojevi

Mreža ima dva skrivena sloja, jedan dimenzije 128 neurona i drugi dimenzije 64 neurona. Oba sloja kao aktivaciju koriste *ReLu* funkciju.



#### Izlazni sloj

Izlazni sloj mreže ima 10 neurona. Svaki od njih odgovara jednoj od cifara od 0 do 9. Ideja je da vrednosti izlaza predstavljaju verovatnoće pripadanja odgovarajućoj klasi cifara. Preciznije, ako se za ulaz *img* koji predstavlja sliku dobija izlaz oblika $(v_0, v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8, v_9)$ rezultat tj. klasa pripadanja će biti ona vrednost $i$ za koju je $v_i$ maksimalno. Na primer, ako $v_4$ ima najveću vrednost, onda zaključujemo da slika *img*  predstavlja broj 4. 

Zato ćemo u poslednjem sloju mreže kao aktivacionu funkciju iskoristiti funkciju mekog maksimuma (engl. softmax) koja vrednost $v_i$ mapira u vrednost $\frac{e^{v_i}}{\sum_j{e^{v_j}}}$

U zadacima višeklasne klasifikacije, uz aktivaciju mekog maksimuma u zadanjem sloju, obično se kao funkcija gubitka koristi `kategorička unakrsna entropija` (engl. categorical crossentropy). Ona predstavlja uopštenje binarne unakrsne entropije i računa se po formuli $-\sum_i{y_i \cdot log(\hat{y_i})}$ u kojoj $\hat{y_i}$ predstavlja predikciju mreže za instacu $i$, a $y_i$ njenu stvarnu vrednost. 

Kao optimizator ćemo koristiti `Adam`. 

Pratićemo tačnost kao relevantnu metriku za ocenu klasifikacije.

Iz grafika funkcije greške zaključujemo da dolazi do preprilagođavanja modela. 

U opštem slučaju, prilikom analize tačnosti klasifikacije možemo se voditi zaključcima prikazanim na slici. <img src='assets/train_val_accuracy.png'>

Jedan način da se doda regularizacija modelu je uz korišćenje L2 regularizacije na nivou slojeva mreže.

Zbog velikog broja parametara, regularizacije se gotovo pa neizostavno koriste u procesu obučavanja neuronskih mreža. Još jedna tehnika regularizacije je takozvana `dropout` regularizacija. Korišćenjem `dropout` regularizacije isključujemo nasumično odabrane neurone, omogućavamo drugačiji protok podataka kroz mrežu i smanjujemo zavisnosti izlaza od ulaza. Da bi se primenila ova tehnika regularizacije, dovoljno je mreži dodati `Dropout` sloj. Njegov parametar `rate` je verovatnoća isključivanja pojedinačnih neurona.
<img src='assets/dropout.png'>

### Korak 4: Evaluacija modela

Model ćemo oceniti na skupu za testiranje.

### Korak 5: Analiza prostora grešaka

Videli smo da model daje dobru ocenu tačnosti. U praksi je značajno izdvojiti i instance na kojima model ne radi dobro u cilju boljeg razumevanja njegovog rada i budućih popravki.

Možemo nacrtati, na primer, prvih 20 instanci test skupa i obeležja koja im pridružuje klasifikator. 

In [ ]:
fig = plt.figure(figsize=(20, 16))
columns = 4
rows = 5
for i in range(1, rows*columns +1):
    # kreiramo blok u okviru kojeg ćemo prikazati sliku
    fig.add_subplot(rows, columns, i)

    # dohvatamo odgovarajući vektor slike i transformišemo ga u 2D sliku dimenzija 28x28
    img = X_test[i].reshape(image_size, image_size)
    
    # postavljamo u bloku naslov koji sadrzi tacnu vrednosti i predviđenu vrednosti
    true_label = np.argmax(y_test[i])
    predicted_label = np.argmax(y_predictions[i])
    plt.title('Ocekivana klasa {0} \nPridruzena klasa: {1}'.format(true_label, predicted_label))
    plt.axis('off')
    
    # prikazujemo sliku
    plt.imshow(img, cmap='gray')
    
plt.show()

### Korak 6: Finalni model

Mreža se sada može istrenirati na celom skupu za učenje, a zatim i oceniti na skupu za testiranje. U praksi se ipak, zbog veličine skupova podataka i zahtevnosti treniranja, za finalni model proglašava model obučen na skupu za treniranje.

In [ ]:
model_final = Sequential([
    InputLayer(input_shape=(image_size*image_size,)),
    Dense(units=128, activation='relu'), 
    Dropout(rate=0.4),
    Dense(units=64, activation='relu'),
    Dropout(rate=0.3),
    Dense(units=number_of_classes, activation='softmax')
])

In [ ]:
model_final.compile(loss=CategoricalCrossentropy(), optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [ ]:
history_final = model_final.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

In [ ]:
test_loss, test_accuracy = model_final.evaluate(X_test, y_test)
print ("Test loss: {}, test accuracy: {}".format(test_loss, test_accuracy))

In [ ]:
train_loss, train_accuracy = model_final.evaluate(X_train, y_train)
print ("Train loss: {}, train accuracy: {}".format(train_loss, train_accuracy))